In [3]:
from data_tokenizing import tokenize_data
from data_nlp import show_nlp_results
from sklearn import svm
from sklearn.metrics import classification_report
import time

In [4]:
(x_train, y_train, x_test, y_test, _, _) = tokenize_data()
print(len(x_train[0]), len(x_test[0]))

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/daniel.chancir/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/daniel.chancir/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/daniel.chancir/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/Users/daniel.chancir/Documents/GitHub/aprendizaje_maquinas/data_cleaning.py:45: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


KeyboardInterrupt: 

In [ ]:
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
print('inicio')
classifier_linear.fit(x_train, y_train)
print('termina el entrenamiento')
t1 = time.time()
prediction_linear = classifier_linear.predict(x_test)
print('terminé de predecir papito')
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

inicio


In [ ]:
# results
print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(y_test, prediction_linear, output_dict=True)